In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
df_country_wise = spark.read.csv("/FileStore/tables/country_wise_latest.csv",header=True,inferSchema=True)
df_clean_complete = spark.read.csv("/FileStore/tables/covid_19_clean_complete.csv")
df_day_wise = spark.read.csv("/FileStore/tables/day_wise.csv")
df_full_group = spark.read.csv("/FileStore/tables/full_grouped.csv")
df_world_data = spark.read.csv("/FileStore/tables/worldometer_data.csv")

In [0]:


df_country_wise.select("Country/Region","Confirmed").orderBy("Confirmed",ascending=False).show(10)

+--------------+---------+
|Country/Region|Confirmed|
+--------------+---------+
|            US|  4290259|
|        Brazil|  2442375|
|         India|  1480073|
|        Russia|   816680|
|  South Africa|   452529|
|        Mexico|   395489|
|          Peru|   389717|
|         Chile|   347923|
|United Kingdom|   301708|
|          Iran|   293606|
+--------------+---------+
only showing top 10 rows



In [0]:
#display(df_country_wise)
df_country_wise.select("Country/Region","Confirmed","Deaths","Recovered","Active").agg({"Confirmed": "sum",
                                                                                                                 "Deaths":"sum",
                                                                                                                 "Recovered" : "sum",
                                                                                                                 "Active":"sum"}).show(10)

+--------------+-----------+-----------+--------------+
|sum(Recovered)|sum(Active)|sum(Deaths)|sum(Confirmed)|
+--------------+-----------+-----------+--------------+
|       9468087|    6358362|     654036|      16480485|
+--------------+-----------+-----------+--------------+



In [0]:

df_country_wise.select("Country/Region","Deaths / 100 Cases").orderBy("Deaths / 100 Cases",ascending=False).show(10)


+--------------+------------------+
|Country/Region|Deaths / 100 Cases|
+--------------+------------------+
|         Yemen|             28.56|
|United Kingdom|             15.19|
|       Belgium|             14.79|
|         Italy|             14.26|
|        France|             13.71|
|       Hungary|              13.4|
|   Netherlands|             11.53|
|        Mexico|             11.13|
|         Spain|             10.44|
|Western Sahara|              10.0|
+--------------+------------------+
only showing top 10 rows



In [0]:
#display(df_country_wise)
#df_country_wise = df_country_wise.withColumnRenamed("New cases","New_cases")
window_spec = Window.orderBy(df_country_wise["New_cases"].desc())
df_country_wise.withColumn("rank_column", rank().over(window_spec)).show()

+--------------+---------+------+---------+-------+---------+----------+-------------+------------------+---------------------+----------------------+-------------------+-------------+-----------------+--------------------+-----------+
|Country/Region|Confirmed|Deaths|Recovered| Active|New_cases|New deaths|New recovered|Deaths / 100 Cases|Recovered / 100 Cases|Deaths / 100 Recovered|Confirmed last week|1 week change|1 week % increase|          WHO Region|rank_column|
+--------------+---------+------+---------+-------+---------+----------+-------------+------------------+---------------------+----------------------+-------------------+-------------+-----------------+--------------------+-----------+
|            US|  4290259|148011|  1325804|2816444|    56336|      1076|        27941|              3.45|                 30.9|                 11.16|            3834677|       455582|            11.88|            Americas|          1|
|         India|  1480073| 33408|   951166| 495499|    4

In [0]:
df_country_wise.printSchema()

root
 |-- Country/Region: string (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- Active: integer (nullable = true)
 |-- New_cases: integer (nullable = true)
 |-- New deaths: integer (nullable = true)
 |-- New recovered: integer (nullable = true)
 |-- Deaths / 100 Cases: double (nullable = true)
 |-- Recovered / 100 Cases: double (nullable = true)
 |-- Deaths / 100 Recovered: string (nullable = true)
 |-- Confirmed last week: integer (nullable = true)
 |-- 1 week change: integer (nullable = true)
 |-- 1 week % increase: double (nullable = true)
 |-- WHO Region: string (nullable = true)

